In [6]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-7043.csv")

In [7]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


### START ### # RANK DT method ##########
num_df_only=num_df.rank(pct=True)
num_df_only=num_df_only*100
num_df_step2=num_df_only
#################################
num_df_step2[num_df_step2 <= 10] = 1
num_df_step2[(num_df_step2 > 10) & (num_df_step2 <=20)]=2
num_df_step2[(num_df_step2 > 20) & (num_df_step2 <=30)]=3
num_df_step2[(num_df_step2 > 30) & (num_df_step2 <=40)]=4
num_df_step2[(num_df_step2 > 40) & (num_df_step2 <=50)]=5
num_df_step2[(num_df_step2 > 50) & (num_df_step2 <=60)]=6
num_df_step2[(num_df_step2 > 60) & (num_df_step2 <=70)]=7
num_df_step2[(num_df_step2 > 70) & (num_df_step2 <=80)]=8
num_df_step2[(num_df_step2 > 80) & (num_df_step2 <=90)]=9
num_df_step2[(num_df_step2 > 90)]=10

### END ### # RANK DT method ##########
 

result_df = pd.concat([num_df_step2, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [8]:
X.shape

(7043, 19)

In [9]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
18,TotalCharges,1616.483021
1,tenure,1389.673822
15,Contract,1115.780167
9,OnlineSecurity,551.611529
12,TechSupport,523.303866
2,MonthlyCharges,357.623600
10,OnlineBackup,230.086520
11,DeviceProtection,191.303140
5,Dependents,133.036443
16,PaperlessBilling,105.680863


In [10]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(7043, 15)

In [11]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3952 1222 493 1376
pod:  0.7362225789192082
pof:  0.23618090452261306
AUC:  0.7500208371982976


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits
{'var_smoothing': 8.111308307896872e-07}
GaussianNB(var_smoothing=8.111308307896872e-07)
0.7944053514205217


In [13]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=3.5111917342151277e-07)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4288 886 662 1207
pod:  0.6457998929909042
pof:  0.17124081948202552
AUC:  0.7372795367544394
accuracy:  0.7802072980264092


In [14]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'C': 8, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=8, multi_class='ovr', penalty='l1', solver='liblinear')
0.8056235896196002


In [16]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4672 502 871 998
pod:  0.5339753879079722
pof:  0.09702357943563973
AUC:  0.7184759042361663
accuracy:  0.8050546642055942


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 10 folds for each of 16 candidates, totalling 160 fits
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance')
0.7570641924564797


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='distance', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4897 277 1434 435
pod:  0.23274478330658105
pof:  0.05353691534596057
AUC:  0.5896039339803102
accuracy:  0.7570637512423684


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
{'bootstrap': True, 'max_depth': 80, 'max_features': 2, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 200}
RandomForestClassifier(max_depth=80, max_features=2, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=200)
0.8047705109606706


In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4736 438 951 918
pod:  0.4911717495987159
pof:  0.08465403942790878
AUC:  0.7032588550854036
accuracy:  0.8027829050120687


In [21]:
# Decision Tree
import time
start = time.time()
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
{'criterion': 'gini', 'splitter': 'best'}
DecisionTreeClassifier()
0.738038362346873
Time:  0.21166110038757324


In [22]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier =DecisionTreeClassifier(criterion='entropy', splitter='random')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4178 996 904 965
pod:  0.516318887105404
pof:  0.1925009663703131
AUC:  0.6619089603675454
accuracy:  0.7302285957688485


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2090909090909091, 'n_estimators': 10, 'subsample': 0.95}
GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.2090909090909091,
                           n_estimators=10, subsample=0.95)
0.7936953981302385


In [25]:
#  Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier =GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.2090909090909091,
                           n_estimators=10, subsample=0.95)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4880 294 1188 681
pod:  0.36436597110754415
pof:  0.05682257441051411
AUC:  0.653771698348515
accuracy:  0.7895783046997018


In [ ]:
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        n_cols = X2_train.shape[1]
        nn = Sequential()
        nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        nn.add(Dense(250, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return nn
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

In [26]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

import time
start = time.time()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs= 100, batch_size= 20, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/100
254/254 [==============================] - 17s 6ms/step - loss: 13.1936 - accuracy: 0.6221 - val_loss: 1.3790 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9682 - accuracy: 0.6660 - val_loss: 0.6086 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7728 - accuracy: 0.6679 - val_loss: 0.6946 - val_accuracy: 0.5063
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6359 - accuracy: 0.6993 - val_loss: 0.5917 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6145 - accuracy: 0.7114 - val_loss: 0.6056 - val_accuracy: 0.7287
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5868 - accuracy: 0.7285 - val_loss: 0.6213 - val_accuracy: 0.7240
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5963 - accuracy: 0.7327 - val_loss: 0.5841 - val_accuracy: 0.72

254/254 [==============================] - 2s 4ms/step - loss: 3.0506 - accuracy: 0.6245 - val_loss: 0.6413 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7214 - accuracy: 0.6671 - val_loss: 0.7392 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5793 - accuracy: 0.7165 - val_loss: 0.4769 - val_accuracy: 0.7437
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5648 - accuracy: 0.7273 - val_loss: 0.4898 - val_accuracy: 0.7760
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.4939 - accuracy: 0.7527 - val_loss: 0.5747 - val_accuracy: 0.7303
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.4937 - accuracy: 0.7518 - val_loss: 0.4772 - val_accuracy: 0.7587
y2_pred:  [[2.12866068e-02]
 [1.74860835e-01]
 [3.38463038e-01]
 [9.39802527e-02]
 [2.12276936e-01]
 [1.03132486e-01]
 [1.64965540e-01]
 [4.11726505e-01]


254/254 [==============================] - 2s 4ms/step - loss: 12.5989 - accuracy: 0.6143 - val_loss: 0.6004 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.8756 - accuracy: 0.6667 - val_loss: 0.6103 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7005 - accuracy: 0.6944 - val_loss: 0.5812 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6579 - accuracy: 0.7052 - val_loss: 0.5843 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6304 - accuracy: 0.7129 - val_loss: 0.6530 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6049 - accuracy: 0.7244 - val_loss: 0.6000 - val_accuracy: 0.7279
y2_pred:  [[2.1041748e-01]
 [2.3199910e-01]
 [2.8802127e-01]
 [1.7744240e-01]
 [2.2191224e-01]
 [1.7050147e-01]
 [2.3081699e-01]
 [1.5124524e-01]
 [1.731

254/254 [==============================] - 2s 4ms/step - loss: 5.9362 - accuracy: 0.6187 - val_loss: 0.6307 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7517 - accuracy: 0.6840 - val_loss: 0.5467 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6067 - accuracy: 0.7108 - val_loss: 0.5522 - val_accuracy: 0.7421
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5561 - accuracy: 0.7322 - val_loss: 0.8215 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5504 - accuracy: 0.7431 - val_loss: 0.4613 - val_accuracy: 0.7879
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5278 - accuracy: 0.7335 - val_loss: 0.5176 - val_accuracy: 0.7326
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.4850 - accuracy: 0.7554 - val_loss: 0.4874 - val_accuracy: 0.7524
Epoch 8/100

254/254 [==============================] - 2s 5ms/step - loss: 8.3007 - accuracy: 0.5951 - val_loss: 0.6282 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9226 - accuracy: 0.6444 - val_loss: 0.5869 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7178 - accuracy: 0.6774 - val_loss: 0.5492 - val_accuracy: 0.7287
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6358 - accuracy: 0.7088 - val_loss: 0.5850 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5944 - accuracy: 0.7212 - val_loss: 0.5657 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5566 - accuracy: 0.7348 - val_loss: 0.7619 - val_accuracy: 0.7279
y2_pred:  [[9.04154480e-02]
 [2.44678020e-01]
 [1.10571384e-01]
 [6.50924444e-03]
 [2.86978483e-03]
 [2.40346193e-02]
 [8.27133656e-03]
 [2.51927972e-03]


254/254 [==============================] - 2s 4ms/step - loss: 10.5562 - accuracy: 0.6244 - val_loss: 0.8518 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9203 - accuracy: 0.6641 - val_loss: 1.2140 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6992 - accuracy: 0.6803 - val_loss: 0.5874 - val_accuracy: 0.7287
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5864 - accuracy: 0.7216 - val_loss: 0.6797 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5575 - accuracy: 0.7342 - val_loss: 0.5942 - val_accuracy: 0.6443
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5404 - accuracy: 0.7308 - val_loss: 0.4683 - val_accuracy: 0.7744
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5621 - accuracy: 0.7332 - val_loss: 0.4997 - val_accuracy: 0.7295
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 10.4783 - accuracy: 0.6003 - val_loss: 0.9169 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9345 - accuracy: 0.6583 - val_loss: 1.8328 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7765 - accuracy: 0.6844 - val_loss: 0.5929 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6002 - accuracy: 0.7359 - val_loss: 0.5831 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5969 - accuracy: 0.7257 - val_loss: 0.7221 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5993 - accuracy: 0.7281 - val_loss: 0.6230 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5880 - accuracy: 0.7333 - val_loss: 0.5856 - val_accuracy: 0.7279
y2_pred:  

254/254 [==============================] - 2s 4ms/step - loss: 12.8641 - accuracy: 0.6186 - val_loss: 1.3138 - val_accuracy: 0.3068
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.1666 - accuracy: 0.6302 - val_loss: 0.5955 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6844 - accuracy: 0.6872 - val_loss: 0.7546 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6507 - accuracy: 0.7182 - val_loss: 0.6005 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6302 - accuracy: 0.7099 - val_loss: 0.5876 - val_accuracy: 0.7287
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6328 - accuracy: 0.7070 - val_loss: 0.5870 - val_accuracy: 0.7287
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6020 - accuracy: 0.7332 - val_loss: 0.5807 - val_accuracy: 0.7279
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 11.2766 - accuracy: 0.6139 - val_loss: 0.6137 - val_accuracy: 0.7248
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.8284 - accuracy: 0.6720 - val_loss: 0.9295 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6320 - accuracy: 0.7185 - val_loss: 0.5908 - val_accuracy: 0.7248
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6061 - accuracy: 0.7300 - val_loss: 0.5903 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6002 - accuracy: 0.7407 - val_loss: 0.5873 - val_accuracy: 0.7248
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5748 - accuracy: 0.7405 - val_loss: 0.5857 - val_accuracy: 0.7248
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5686 - accuracy: 0.7425 - val_loss: 0.5878 - val_accuracy: 0.7248
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 8.3840 - accuracy: 0.6238 - val_loss: 2.9768 - val_accuracy: 0.2973
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 2.9185 - accuracy: 0.6019 - val_loss: 0.7048 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9075 - accuracy: 0.6801 - val_loss: 0.6267 - val_accuracy: 0.7256
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6477 - accuracy: 0.7157 - val_loss: 0.5617 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6215 - accuracy: 0.7037 - val_loss: 0.6905 - val_accuracy: 0.5008
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6445 - accuracy: 0.7153 - val_loss: 0.5779 - val_accuracy: 0.7248
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6043 - accuracy: 0.7205 - val_loss: 0.5462 - val_accuracy: 0.7256
Epoch 8/100

In [ ]:
##RNN Hyperparemeter tuning
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    #n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        #n_cols = X2_train.shape[1]
        #nn = Sequential()
        #nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        #nn.add(Dense(250, activation='relu'))
        #nn.add(Dense(1, activation='sigmoid'))
        #nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model = Sequential()
        model.add(Embedding(10000, 32))
        model.add(LSTM(32))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
        return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

In [27]:
#RNN
import time
start = time.time()

def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/5
40/40 [==============================] - 6s 34ms/step - loss: 0.6195 - acc: 0.7386 - val_loss: 0.5379 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5218 - acc: 0.7354 - val_loss: 0.5151 - val_acc: 0.7271
Epoch 3/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4975 - acc: 0.7423 - val_loss: 0.5234 - val_acc: 0.7326
Epoch 4/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4880 - acc: 0.7513 - val_loss: 0.5051 - val_acc: 0.7350
Epoch 5/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4773 - acc: 0.7557 - val_loss: 0.5099 - val_acc: 0.7397
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 3s 39ms/step - loss: 0.6175 - acc: 0.7103 - val_loss: 0.5473 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 18ms/step - loss: 0.5150 - acc: 0.7443 - val_loss: 0.5153 - val_acc: 0.7303
Epoch 3/5
40/40 [==============================] - 1s 18ms/step - loss: 0.4997 - acc: 0.7309 - val_loss: 0.5088 - val_acc: 0.7350
Epoch 4/5
40/40 [==============================] - 1s 18ms/step - loss: 0.4872 - acc: 0.7470 - val_loss: 0.5147 - val_acc: 0.7382
Epoch 5/5
40/40 [==============================] - 1s 18ms/step - loss: 0.4777 - acc: 0.7537 - val_loss: 0.5238 - val_acc: 0.7342
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 3s 30ms/step - loss: 0.6121 - acc: 0.7054 - val_loss: 0.5410 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5315 - acc: 0.7274 - val_loss: 0.5194 - val_acc: 0.7358
Epoch 3/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4988 - acc: 0.7545 - val_loss: 0.5139 - val_acc: 0.7303
Epoch 4/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5054 - acc: 0.7288 - val_loss: 0.5164 - val_acc: 0.7374
Epoch 5/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4766 - acc: 0.7608 - val_loss: 0.5072 - val_acc: 0.7374
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 3s 31ms/step - loss: 0.6173 - acc: 0.7263 - val_loss: 0.5407 - val_acc: 0.7279
Epoch 2/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5203 - acc: 0.7423 - val_loss: 0.5220 - val_acc: 0.7374
Epoch 3/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4972 - acc: 0.7462 - val_loss: 0.5147 - val_acc: 0.7263
Epoch 4/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4861 - acc: 0.7451 - val_loss: 0.5063 - val_acc: 0.7350
Epoch 5/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4815 - acc: 0.7514 - val_loss: 0.5045 - val_acc: 0.7334
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Epoch 1/5
40/40 [==============================] - 3s 31ms/step - loss: 0.6165 - acc: 0.6956 - val_loss: 0.5422 - val_acc: 0.7248
Epoch 2/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5329 - acc: 0.7251 - val_loss: 0.5215 - val_acc: 0.7232
Epoch 3/5
40/40 [==============================] - 1s 17ms/step - loss: 0.5020 - acc: 0.7349 - val_loss: 0.5126 - val_acc: 0.7208
Epoch 4/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4905 - acc: 0.7503 - val_loss: 0.5106 - val_acc: 0.7232
Epoch 5/5
40/40 [==============================] - 1s 17ms/step - loss: 0.4847 - acc: 0.7472 - val_loss: 0.5152 - val_acc: 0.7263
y22_pred:  [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0